In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from pytrends import dailydata
import requests
import collections

from sklearn import linear_model
import statsmodels.api as sm

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
with open("words.txt") as f:
    words = f.readlines()
words = [x.strip() for x in words]
words = words[:5] + ["debt"]
words

['hedge', 'dividend', 'earnings', 'inflation', 'markets', 'debt']

In [4]:
length = 103
metric = 0

profits = {} # keeps track of words with best profit
word_types = collections.defaultdict(list) # keeps track of which words have positive vs negative sentiment

In [5]:
def get_trend(key_word, start_year, end_year, start_month, end_month, profits, word_types):
    trend = pd.DataFrame(dailydata.get_daily_data(key_word, start_year, start_month, end_year, end_month, wait_time = 0)[key_word])
    trend["change"] = trend.pct_change()
    
    # grab DJIA data
    djia = yf.download("DJIA", start = str(start_year)+"-"+str(start_month)+"-01", end = str(end_year)+"-"+str(end_month)+"-01")["Adj Close"]
    
    # join trend data with DJIA data
    joined = trend.merge(djia, left_on = trend.index, right_on = djia.index)
    joined = joined.rename(columns = {"key_0": "Date"})
    
    # grab the adj close price difference for each day
    joined['Diff'] = joined['Adj Close'].diff()
    
    
    # we implement the algorithm described, where if the change is greater than some metric,
    # then we sell at the current price (assume we own at start) and buy the following week at that price,
    # this then corresponds to subtracting the difference between week i+1 and week i (we hope price decreased, leading to positive gain)
    # we do the opposite for the less than case
    # NOTE: assumes that these words have negative sentiment, ie that increased search volume will lead to a decrease in price
    
    def getVal(row):
        if row.name < length-1 and joined['change'].iloc[row.name+1] > metric:
            return -1 * joined['Diff'].iloc[row.name+1]
        elif row.name < length-1:
            return joined['Diff'].iloc[row.name+1]
        else:
            return 0
        
    # determine whether to add or subtract the difference
    # if the % change is > 0 (or some arbitrary metric), then we know to sell the asset and rebuy at the next week
    # if % change is < 0, then we buy the asset and resell it the next week
    joined['Recent Profit'] = joined.apply(getVal, axis = 1)
    
    # get cumulative returns
    joined['Total Profit'] = joined['Recent Profit'].cumsum()
    
    # record final profit (may be slightly off due to buying/selling at beginning/end)
    profits[key_word] = max(joined.iloc[-1]['Total Profit'], -1 * joined.iloc[-1]['Total Profit'])
    
    if profits[key_word] == joined.iloc[-1]['Total Profit']: # negative sentiment word, naive classification
        word_types['Negative'].append(key_word)
    else:
        word_types['Positive'].append(key_word)
        
    
    return joined

In [6]:
word_trends = {}    #dictionary with key=word, value=DataFrame with Google trend and pct change 
for word in words:
    word_trends[word] = get_trend(word, 2014, 2014, 1, 6, profits, word_types)

hedge:2014-01-01 2014-01-31
hedge:2014-02-01 2014-02-28
hedge:2014-03-01 2014-03-31
hedge:2014-04-01 2014-04-30
hedge:2014-05-01 2014-05-31
hedge:2014-06-01 2014-06-30
[*********************100%***********************]  1 of 1 completed
dividend:2014-01-01 2014-01-31
dividend:2014-02-01 2014-02-28
dividend:2014-03-01 2014-03-31
dividend:2014-04-01 2014-04-30
dividend:2014-05-01 2014-05-31
dividend:2014-06-01 2014-06-30
[*********************100%***********************]  1 of 1 completed
earnings:2014-01-01 2014-01-31
earnings:2014-02-01 2014-02-28
earnings:2014-03-01 2014-03-31
earnings:2014-04-01 2014-04-30
earnings:2014-05-01 2014-05-31
earnings:2014-06-01 2014-06-30
[*********************100%***********************]  1 of 1 completed
inflation:2014-01-01 2014-01-31
inflation:2014-02-01 2014-02-28
inflation:2014-03-01 2014-03-31
inflation:2014-04-01 2014-04-30
inflation:2014-05-01 2014-05-31
inflation:2014-06-01 2014-06-30
[*********************100%***********************]  1 of 1 co

In [7]:
profits

{'hedge': 1029.36328125,
 'dividend': 1694.595703125,
 'earnings': 702.658203125,
 'inflation': 2067.701171875,
 'markets': 984.505859375,
 'debt': 197.62109375}

In [8]:
word_types

defaultdict(list,
            {'Positive': ['hedge', 'dividend', 'earnings', 'inflation'],
             'Negative': ['markets', 'debt']})

In [9]:
word_trends["debt"]

,Date,debt,change,Adj Close,Diff,Recent Profit,Total Profit
0,2014-01-02,34.68,1.020979,16441.349609,NaN,28.640625,28.640625
1,2014-01-03,34.68,0.000000,16469.990234,28.640625,44.890625,73.531250
2,2014-01-06,35.51,0.988242,16425.099609,-44.890625,-105.839844,-32.308594
3,2014-01-07,44.40,0.250352,16530.939453,105.839844,68.199219,35.890625
4,2014-01-08,47.79,0.076351,16462.740234,-68.199219,-17.980469,17.910156
...,...,...,...,...,...,...,...
98,2014-05-23,38.25,-0.235764,16606.269531,63.189453,-69.230469,287.072266
99,2014-05-27,47.40,2.340381,16675.500000,69.230469,-42.320312,244.751953
100,2014-05-28,43.68,-0.078481,16633.179688,-42.320312,-65.560547,179.191406
101,2014-05-29,46.74,0.070055,16698.740234,65.560547,18.429688,197.621094


## Old Stuff

In [10]:
fortune_500 = pd.read_csv("Fortune 500 2017 - Fortune 500.csv")
fortune_500.head()

,Rank,Title,Website,Employees,Sector,Industry,Hqlocation,Hqaddr,Hqcity,Hqstate,Hqzip,Hqtel,Ceo,Ceo-title,Address,Ticker,Fullname,Revenues,Revchange,Profits,Prftchange,Assets,Totshequity
0,1,Walmart,http://www.walmart.com,2300000,Retailing,General Merchandisers,"Bentonville, AR",702 S.W. Eighth St.,Bentonville,AR,72716,479-273-4000,C. Douglas McMillon,"President, Chief Executive Officer & Director","702 S.W. Eighth St., Bentonville, AR 72716",WMT,"Wal-Mart Stores, Inc.",485873,0.8,13643.0,-7.2,198825,77798.0
1,2,Berkshire Hathaway,http://www.berkshirehathaway.com,367700,Financials,Insurance: Property and Casualty (Stock),"Omaha, NE",3555 Farnam St.,Omaha,NE,68131,402-346-1400,Warren E. Buffett,Chairman & Chief Executive Officer,"3555 Farnam St., Omaha, NE 68131",BRKA,Berkshire Hathaway Inc.,223604,6.1,24074.0,0,620854,283001.0
2,3,Apple,http://www.apple.com,116000,Technology,"Computers, Office Equipment","Cupertino, CA",1 Infinite Loop,Cupertino,CA,95014,408-996-1010,Timothy D. Cook,Chief Executive Officer & Director,"1 Infinite Loop, Cupertino, CA 95014",AAPL,"Apple, Inc.",215639,-7.7,45687.0,-14.4,321686,128249.0
3,4,Exxon Mobil,http://www.exxonmobil.com,72700,Energy,Petroleum Refining,"Irving, TX",5959 Las Colinas Blvd.,Irving,TX,75039,972-444-1000,Darren W. Woods,Chairman & Chief Executive Officer,"5959 Las Colinas Blvd., Irving, TX 75039",XOM,Exxon Mobil Corporation,205004,-16.7,7840.0,-51.5,330314,167325.0
4,5,McKesson,http://www.mckesson.com,68000,Wholesalers,Wholesalers: Health Care,"San Francisco, CA",1 Post St.,San Francisco,CA,94104,415-983-8300,John H. Hammergren,"Chairman, President & Chief Executive Officer","1 Post St., San Francisco, CA 94104",MCK,McKesson Corporation,192487,6.2,2258.0,53,56563,8924.0


In [11]:
def get_name(symbol):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(symbol)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
        if x['symbol'] == symbol:
            return x['name']

In [12]:
def get_data(ticker: str):
    print("Stock data for: \"" + str(ticker) + "\"")
    data = pd.DataFrame(yf.download(ticker, start = '2020-1-1', end = "2020-6-1", interval = "1d")["Adj Close"])
    name = fortune_500[fortune_500["Ticker"] == ticker][["Title"]].iloc[0][0]
    print("Search trends for: \"" + str(name) + "\"")
    trend = dailydata.get_daily_data(name, 2020, 1, 2020, 6, wait_time = 0)[name]
    data.index.name = "date"
    data = data.join(trend, how = "inner")
    return data

In [13]:
df = get_data("TSLA")
df.head(50)

Stock data for: "TSLA"
[*********************100%***********************]  1 of 1 completed
Search trends for: "Tesla"
Tesla:2020-01-01 2020-01-31
Tesla:2020-02-01 2020-02-29
Tesla:2020-03-01 2020-03-31
Tesla:2020-04-01 2020-04-30
Tesla:2020-05-01 2020-05-31
Tesla:2020-06-01 2020-06-30


,Adj Close,Tesla
date,,
2020-01-02,86.052002,21.96
2020-01-03,88.601997,23.94
2020-01-06,90.307999,15.60
2020-01-07,93.811996,22.94
2020-01-08,98.428001,23.18
2020-01-09,96.267998,23.68
2020-01-10,95.629997,19.14
2020-01-13,104.972000,24.96
2020-01-14,107.584000,27.20
